# Introduction en Deep Learning 

### - Ce notebook consiste à classifier des images d’habits. Le dataset est connu sous le nom de Fashion Mnist (un peu du genre MNIST).



Données

Les données à classer sont celles de Fashion-MNIST.

    Fashion-MNIST is a dataset of Zalando‘s article images—consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28×28 grayscale image, associated with a label from 10 classes. We intend Fashion-MNIST to serve as a direct drop-in replacement for the original MNIST dataset for benchmarking machine learning algorithms. It shares the same image size and structure of training and testing splits.
    https://github.com/zalandoresearch/fashion-mnist

### - On utilise la sur-couche Keras à TensorFlow. 

In [ ]:
# Importer TensorFlow 
import tensorflow as tf
from tensorflow import keras 

# Helper libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print(tf.__version__) #version du TF


In [ ]:
print(keras.__version__)

On peut télécharger diredctement le dataset Fashion_mnist depuis keras.dataset.  

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()




La première colonne représente l'étiquette et qu'elle comporte 10 classes, de 0 à 9. Les colonnes restantes sont les données de pixels réelles. Chaque ligne est une représentation d'une image différente sous la forme de données de pixels.


In [ ]:
train_images

In [ ]:
train_images.shape

## Préparation des données

Les images sont en niveaux de gris. Elles ont des valeurs entre 0 et 255, comme on peut le voir avec cet exemple : 

In [ ]:
plt.figure()
plt.imshow(train_images[1].reshape((28,28)))#reshape en image : tableau en matrice
plt.colorbar()
plt.grid(False)
plt.show()


Les étiquettes des données sont des catégories numériques, dont les noms sont :

Label	   Description

0          T-shirt/top

1	       Trouser

2	       Pullover

3	       Dress

4	       Coat

5	       Sandal

6	       Shirt

7	       Sneaker

8	       Bag

9	       Ankle boot

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']



## Affichage des 25 premières images (apprentissage) avec leurs labels

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i].reshape((28,28)), cmap=plt.cm.binary)
    #plt.xlabel(class_names[train_labels[i]])
    plt.xlabel(class_names[train_labels[i].astype(np.int)])
plt.show()


## Construction du modèle (architecture du réseau de neurones)

In [ ]:
train_images.shape

In [ ]:
train_images

In [ ]:
train_images = train_images.reshape(60000,28,28)

In [ ]:
test_images = test_images.reshape(10000,28,28)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [ ]:
## Pour un problème de classification multi-classe

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:

## Pour des métriques sur-mesure
import keras.backend as K

def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', mean_pred])

###Lire la suite sur: https://www.actuia.com/keras/debuter-avec-le-modele-sequentiel-de-keras/
               

### - Lancer l'entrainement sur l'ensemble d'apprentissage ,avec 10 itérations , et prendre 20% de l'ensemlble d'apprentissage comme ensemble de validation

#### Garder l'historique de l'apprentissage dans la variable 'history'

In [ ]:
history = model.fit(train_images, train_labels, epochs=10 , validation_split=0.2)

### - Montrer le graphe de l'historique de l'entrainement sur l'ensemble d'apprentissage et de validation

In [ ]:
loss_curve = history.history["loss"]
acc_curve = history.history["accuracy"]

loss_val_curve = history.history["val_loss"]
acc_val_curve = history.history["val_accuracy"]

plt.plot(loss_curve, label="Train")
plt.plot(loss_val_curve, label="Val")
plt.legend(loc='upper left')
plt.title("Loss")
plt.show()

plt.plot(acc_curve, label="Train")
plt.plot(acc_val_curve, label="Val")
plt.legend(loc='upper left')
plt.title("Accuracy")
plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

## Inférence du modèle
On peut effectuer des prédiction à l'aide du modèle entrainé

In [ ]:
predictions = model.predict(test_images)


- "predictions" contient les prédicitons des labels de tout l'ensmble de test.
- Si on veut voir la prédiction de la première image de l'ensemble de teste:

In [ ]:
predictions[0]


## Questions :
- Que représente ce résultat ? 
- Comment pourrions nous rendre ce résultat plus lisible?

- La prédiction ici est un tableau de 10 nombres. Chaque cellule 'i' du tableau représente la probabilité (selon le modèle) que l'image en entrée correspond à la classe dont le label = "i". 

- On peut voir quelle est la probabilité maximale du modèle avec argmax:

- Selon le modèle, la première image de l'ensemble de teste consite à une imge de la classe 0 => 'T-shirt/top' .
- On vérifie si c'est une bonne prédiction en allant voir le label de la première image de l'ensemble de teste.

## Plus de visualisation ?

In [ ]:
def plot_image(i, predictions_array, true_label, img):
    predictions_array, true_label, img = predictions_array, true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img.reshape((28,28)), cmap=plt.cm.binary)

    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
        color = 'blue'
    else:
        color = 'red'

    plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label.astype(np.int)]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
    predictions_array, true_label = predictions_array, true_label[i]
    plt.grid(False)
    plt.xticks(range(10))
    plt.yticks([])
    thisplot = plt.bar(range(10), predictions_array, color="#777777")
    plt.ylim([0, 1])
    predicted_label = np.argmax(predictions_array)

    thisplot[predicted_label].set_color('red')
    thisplot[true_label.astype(np.int)].set_color('blue')


In [ ]:
i = 0      #3
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()


In [ ]:
# Montrer les premier X test images, leurs labels prédits, et leurs labels correctes . 
# Colorer les prédictions corrects en bleu et les fausses prédictions en rouge.
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
    plt.subplot(num_rows, 2*num_cols, 2*i+1)
    plot_image(i, predictions[i], test_labels, test_images)
    plt.subplot(num_rows, 2*num_cols, 2*i+2)
    plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()


# Exercice 1:

Avec ce modèle basique on a obtenu 83% du taux d'accuracy. Pourriez vous faire mieux ?
- Essayer de tester avec d'autres modèles en changeant les hyperparamètres (nombre de couches, de neurones, etc.)
- Communiquer moi vos noms, votre modèle, et l'accuracy obtenu sur l'ensemble de teste.

# Exercice 2:
Proposer des modèles convolutifs ou autres pour améliorer l'accuracy sur l'ensemble de teste.

NB. Il est permis de choisir n'importe quelle architecture des modèles standards, ie: Unet, VGG, ...etc

- Communiquer moi vos noms, votre modèle, et l'accuracy obtenu sur l'ensemle de teste.